# Hologram MG Scorer & Memory Management Test

This notebook demonstrates how to:
1.  Initialize the Hologram system with a specific project.
2.  Ingest data (simulated code/text concepts).
3.  Use the `MG Scorer` to analyze semantic coherence and stability.
4.  Manage project memory (save, reset) programmatically.

In [ ]:
import sys
import os
import shutil
from pathlib import Path

# Ensure project root is in sys.path
current_dir = os.getcwd()
project_root = str(Path(current_dir).parent)
if project_root not in sys.path:
    sys.path.append(project_root)

import numpy as np
from hologram.api import Hologram
from hologram.mg_scorer import mg_score, MGScore
from hologram.config import Config
from hologram.global_config import global_project_exists, load_global_config

## 1. Environment & Config Check
Check if a global project configuration exists and load it.

In [ ]:
if global_project_exists():
    print("Global project configuration found.")
    config = load_global_config()
    print("Loaded Global Config:", config.keys())
else:
    print("No global project found. Using defaults.")

print(f"Memory Directory: {Config.storage.MEMORY_DIR}")

## 2. Project Initialization
We will create a temporary test project named `test_mg_project`.

In [ ]:
PROJECT_NAME = "test_mg_project"

# Initialize Hologram for this project
# fast_mode=True skips extensive NLP (like coref) for speed in this demo
holo = Hologram.init(
    project=PROJECT_NAME,
    encoder_mode="minilm",  # Lightweight encoder
    use_gravity=True
)

print(f"Initialized Hologram for project: {holo.project}")

## 3. Data Ingestion
Ingesting some related sentences to form a coherent cluster, and some outliers.

In [ ]:
data_cluster = [
    "The quick brown fox jumps over the dog.",
    "A fast brown fox leaps over a lazy dog.",
    "Foxes are known for jumping over dogs quickly."
]

data_outlier = [
    "Quantum mechanics describes the behavior of subatomic particles."
]

print("Ingesting cluster data...")
for text in data_cluster:
    holo.add_text(glyph_id=f"text:{hash(text)}", text=text)

print("Ingesting outlier data...")
for text in data_outlier:
    holo.add_text(glyph_id=f"text:{hash(text)}", text=text)

# Force a save to ensure persistence
holo.save()
print("Data ingested and saved.")

## 4. MG Scorer Analysis
We will retrieve the vectors for our ingested data and run the MG Scorer.

In [ ]:
# Access internal memory to get vectors for validation
# In a real scenario, you might get these from a retrieval set
all_vectors = []
for cid, concept in holo.field.sim.concepts.items():
    if concept.vector is not None:
        all_vectors.append(concept.vector)

print(f"Analyzing {len(all_vectors)} vectors...")

if all_vectors:
    score: MGScore = mg_score(all_vectors)

    print("\n=== MG Scorer Results ===")
    print(f"Coherence:     {score.coherence:.4f}  (High = tight cluster)")
    print(f"Curvature:     {score.curvature:.4f}  (1.0 = linear)")
    print(f"Entropy:       {score.entropy:.4f}  (Low = simple structure)")
    print(f"Collapse Risk: {score.collapse_risk:.4f} (Composite metric)")
    print(f"Gradient Norm: {np.linalg.norm(score.gradient):.4f}")
else:
    print("No vectors found to analyze.")

## 5. Prompt Evaluation (Sentence/Paragraph Level)
We can also use the MG Scorer to evaluate the coherence and stability of ad-hoc text sequences (e.g., assessing a generated paragraph or a set of sentences) without adding them to the project memory.

We will test three scenarios:
1. **Coherent Paragraph**: A set of logically connected sentences.
2. **Incoherent Paragraph**: A set of random, unrelated sentences.
3. **Single Sentence (Token/Word Level)**: Analyzing the flow of a single sentence.

In [ ]:
def print_mg_analysis(label: str, texts: list[str]):
    print(f"\n--- Analyzing: {label} ---")
    print(f"Input ({len(texts)} segments): {texts[:3]} ..." if len(texts) > 3 else f"Input: {texts}")
    
    # Use holo.score_text helper
    score = holo.score_text(texts)
    
    print(f"  Coherence:     {score.coherence:.4f}")
    print(f"  Curvature:     {score.curvature:.4f}")
    print(f"  Entropy:       {score.entropy:.4f}")
    print(f"  Collapse Risk: {score.collapse_risk:.4f}")

# 1. Coherent Paragraph (split into sentences)
coherent_text = [
    "The system initializes the core module.",
    "Once initialized, the module loads the configuration.",
    "The configuration determines the operational mode.",
    "Finally, the system enters the main event loop."
]
print_mg_analysis("Coherent Flow", coherent_text)

# 2. Incoherent Paragraph
incoherent_text = [
    "The weather is nice today.",
    "Memory leaks are hard to debug.",
    "I should buy some groceries.",
    "The stock market crashed yesterday."
]
print_mg_analysis("Incoherent Flow", incoherent_text)

# 3. Single Sentence (Word Level)
sentence = "Holographic memory systems utilize vector fields for semantic storage."
words = sentence.split()
print_mg_analysis("Single Sentence (Tokens)", words)

## 6. Memory Management: Reset
Demonstrating how to clean up/reset the project memory.

In [ ]:
# Path to project memory
project_path = Path(Config.storage.MEMORY_DIR) / PROJECT_NAME

print(f"checking project path: {project_path}")
if project_path.exists():
    print(f"Removing project memory at {project_path}...")
    shutil.rmtree(project_path)
    print("Project memory reset successfully.")
else:
    print("Project path does not exist (already clean).")

# Verify deletion
print("Exists after delete:", project_path.exists())